### 1. Passo a Passo para o Algoritmo Random Forest (RF)

O algoritmo Random Forest é um método de machine learning que usa uma coleção de árvores de decisão para melhorar a precisão e evitar overfitting. Passo a passo:

1. **Preparação dos Dados:**
   - Divida os dados em conjunto de treinamento e teste.

2. **Bootstrapping:**
   - Para cada árvore na floresta (geralmente várias árvores são usadas):
     - Crie um subconjunto do conjunto de dados de treinamento original usando amostragem com reposição (bootstrap).

3. **Seleção de Features:**
   - Para cada nó da árvore, selecione aleatoriamente um subconjunto de features (variáveis explicativas).
   - Essa seleção aleatória de features é o que diferencia o Random Forest de outros métodos como o Bagging.

4. **Construção da Árvore de Decisão:**
   - Construa uma árvore de decisão usando o subconjunto de dados e features selecionados.
   - Cresça a árvore até o máximo possível sem poda.

5. **Repetição:**
   - Repita os passos acima para criar várias árvores independentes.

6. **Agregação:**
   - Para a predição, agregue as predições de todas as árvores.
   - No caso de problemas de regressão, tire a média das predições.
   - No caso de problemas de classificação, use a votação majoritária para decidir a classe final.

### 2. Explicação do Random Forest

O Random Forest é um algoritmo de aprendizado de máquina baseado em uma coleção de árvores de decisão. A ideia principal é que, ao combinar as previsões de várias árvores de decisão individuais que foram treinadas de forma ligeiramente diferente (graças ao bootstrapping e à seleção aleatória de features), o modelo resultante é mais robusto e geralmente tem melhor desempenho do que qualquer árvore individual. Ele é menos propenso ao overfitting em comparação com uma única árvore de decisão, porque cada árvore vê apenas uma parte dos dados e das features, o que introduz mais diversidade no modelo.



### 3. Diferença entre Bagging e Random Forest

- **Bagging (Bootstrap Aggregating):**
  - Envolve a criação de múltiplos conjuntos de treinamento a partir do conjunto de dados original usando amostragem com reposição (bootstrap).
  - Cada conjunto de treinamento é usado para treinar um modelo (geralmente uma árvore de decisão).
  - As predições desses modelos são então agregadas (por média no caso de regressão ou por votação no caso de classificação).

- **Random Forest:**
  - Usa a mesma técnica de bootstrapping que o Bagging para criar conjuntos de treinamento.
  - Além disso, em cada nó de cada árvore de decisão, uma seleção aleatória de features é escolhida como candidatas para a divisão.
  - Essa seleção aleatória de features em cada nó aumenta a diversidade das árvores e melhora o desempenho do modelo final.

In [4]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [8]:
X = load_iris().data
y = load_iris().target
df = pd.DataFrame(X, columns=load_iris().feature_names)
df['target'] = y

def rf_classifier(df: pd.DataFrame, num_bootstrap_samples: int = 10, test_size: float = 0.33) -> pd.DataFrame:
    
    df_train, df_test = train_test_split(df, test_size=test_size, random_state=42)
    
    X_test = df_test.drop(['target'], axis=1)
    y_test = df_test['target'].rename('y_test')
    
    y_pred_bagging = pd.DataFrame()

    for i in range(num_bootstrap_samples):
        df_train_sample = df_train.sample(n=len(df_train), replace=True, random_state=42+i)
        X_train_sample = df_train_sample.drop(['target'], axis=1)
        
        selected_features = np.random.choice(X_train_sample.columns, size=round(np.sqrt(X_train_sample.shape[1])), replace=False)
        X_train_selected = X_train_sample[selected_features]
        
        y_train_sample = df_train_sample['target']
        
        model = DecisionTreeClassifier(random_state=42+i)
        model.fit(X_train_selected, y_train_sample)
        
        y_pred_bagging[i] = model.predict(X_test[selected_features])
    
    y_pred = y_pred_bagging.mode(axis=1)[0].rename('y_pred')
    
    print('Accuracy score:', accuracy_score(y_true=y_test, y_pred=y_pred))
    
    return pd.concat(objs=[y_test.reset_index(drop=True), y_pred.astype(int)], axis=1)

rf_classifier(num_bootstrap_samples=10, df=df, test_size=0.33)

Accuracy score: 0.96


,y_test,y_pred
0,1,1
1,0,0
2,2,2
3,1,1
4,1,1
5,0,0
6,1,1
7,2,2
8,1,1
9,1,1
